<a href="https://colab.research.google.com/github/joardar-aditya/DeepLearning-1/blob/master/Twitter_Sentiment_Analysis_using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
import numpy as np

In [ ]:
#import nltk tweets dataset 


from nltk.corpus import twitter_samples
from nltk.corpus import stopwords 

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [ ]:
#Creating datatset for the testing and training 
test_set_pos = all_positive_tweets[4000:]
train_set_pos = all_positive_tweets[:4000]
test_set_neg = all_negative_tweets[4000:]
train_set_neg = all_negative_tweets[:4000]

test_x = test_set_pos + test_set_neg
train_x  = train_set_pos + train_set_neg 

test_y = np.append(np.ones((len(test_set_pos), 1)), np.zeros((len(test_set_neg), 1)), axis=0)
train_y = np.append(np.ones((len(train_set_pos), 1)), np.zeros((len(train_set_neg), 1)), axis=0)

In [ ]:
test_x[0]

'Bro:U wan cut hair anot,ur hair long Liao bo\nMe:since ord liao,take it easy lor treat as save $ leave it longer :)\nBro:LOL Sibei xialan'

In [ ]:
''' process tweet -: only stopwords, remove punctuations and for the frequency matrix 
Three important concepts -: 
   1) Tokenizing 
   2) Stemming 
   3) Lower casting '''
import re
import string 

from nltk.stem import PorterStemmer 
from nltk.tokenize import TweetTokenizer 


#Process an individual tweet 

stop_words_english = stopwords.words('english')

def process_tweet(tweet):
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet3 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)
  tweet3 = re.sub(r'#', '', tweet3)
  
  tokenizer = TweetTokenizer(preserve_case =False, strip_handles = True, reduce_len = True)

  tweet_tokens = tokenizer.tokenize(tweet3)
  #Tokenizing Done
  tweet_words = []
  for words in tweet_tokens:
    if (words not in stop_words_english and words not in string.punctuation):
          tweet_words.append(words)

  #Clean Up Done
  tweet_stem = []
  stemmer = PorterStemmer()
  for words in tweet_words:
        stem_word = stemmer.stem(words)
        tweet_stem.append(stem_word)
  #Stemming Done 
  
  return tweet_stem


In [ ]:
#Creating Frequency table
frequency_table = {}

for tweet in test_set_pos + train_set_pos: 
   tweet_stem = process_tweet(tweet)
   for word in tweet_stem:
     if (word, 1) in frequency_table.keys():
         frequency_table[(word,1)] += 1
     else:
         frequency_table[(word,1)] = 1

for tweet in test_set_neg + train_set_neg: 
   tweet_stem = process_tweet(tweet)
   for word in tweet_stem:
     if (word, 0) in frequency_table.keys():
         frequency_table[(word,0)] += 1
     else:
         frequency_table[(word,0)] = 1

In [ ]:
''' PROCESSING FINISHED '''


''' Logistic Regression Implementation '''

' Logistic Regression Implementation '

Gradient Descent Explanation Tweet -: https://twitter.com/joardar7/status/1292144626795593729?s=20



In [ ]:
def sigmoid(z):

  h = 1/(1 + np.exp(-z))

  return h 

no_of_iter = 600 #More iters , more training 

alpha = 0.00001 #Hyperparamter, smaller the better but don't make it too small. 

def gradientdescent(x , y, alpha, theta, no_of_iter):
    
    m = x.shape[0]
    
    # x shape (m, n + 1) and theta  shape (n + 1 , 1) , z shape -: (m , 1), y shape is (m, 1)
    for i in range(no_of_iter):
      
      z = np.dot(x,theta) 

      h = sigmoid(z)

      ''' Cost function is the difference between the predicted value and the actual value of prediction, 
      Now the fun part -: 

      Cost Function for the logistic function is -: 

      J = -1/m * sum( y*log(h) + (1-y)*log(1 - h))

      WHy this is fun, the y is always 0 or 1 so if y = 0  and h is - then y = 0 for first term and log1 = 0 for second which makes error zero. 

      Remember it as the cost function of the logistic regression (every logistic regression)
      
      x remains unchanged 

      
      '''

      J = -1/m * (np.dot(y.transpose(), np.log(h)) + np.dot(1 - y.transpose(),np.log(1 - h))) 
    
     
      theta = theta - alpha/m * np.dot(x.transpose(), (h - y))

    J = float(J)

    return J, theta  
  
      
# make x -: feature set  

def extract_feature(tweet, freq):

  x = np.zeros((1,3))

  ntweet = process_tweet(tweet)

  x[0,0] = 1 #(bias term)

  for word in ntweet :
      
      x[0, 1] += freq.get((word, 1), 0)

      x[0, 2] += freq.get((word, 0), 0)

  return x 











##Training begins -: 

Form the x, get the Y and give the theta (all 0, always has been); 

do gradient descent, and then get the prediction using the theta. 


In [ ]:
X_train = np.zeros((len(train_x), 3))

for i in range(len(train_x)):

  X_train[i, :] = extract_feature(train_x[i], frequency_table) 



J, theta = gradientdescent(X_train, train_y, alpha, np.zeros((3, 1)), no_of_iter)

print(J)
print(f"theta value :{[round(t, 8) for t in np.squeeze(theta)]}")



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in log


nan
theta value :[3.122e-05, 0.0083235, -0.007581]


In [ ]:
def predict_tweet(tweet, theta, freq):
  X = extract_feature(tweet, freq)
  h = sigmoid(np.dot(X, theta))
  return h 

In [ ]:
predict_tweet('I love you', theta, frequency_table)

array([[0.8981759]])

In [ ]:
# Testing accuracy 
correct = 0
for i in range(len(test_x)):

   result = predict_tweet(test_x[i], theta, frequency_table)

   value = 0
   if(float(result) > 0.5):
     value = 1
   if(value == test_y[i]):
     correct += 1

print("Accuracy -: ")

print(correct/len(test_x))


Accuracy -: 
0.9955


# We are happy with a 99.5 % Accuracy! 